Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/automated-machine-learning/classification-credit-card-fraud/auto-ml-classification-credit-card-fraud.png)

# Automated Machine Learning
_**AutoML Model Testing for Classification**_

## Contents
1. [Introduction](#Introduction)
1. [Setup](#Setup)
1. [Data](#Data)
1. [Train](#Train)
1. [Results Analysis and Test](#Results-Analysis-and-Test)
1. [Acknowledgements](#Acknowledgements)

## Introduction

In this example we use the associated credit card dataset to showcase how you can do model training, selection and test with AutoML for classification tasks. Besides this example notebook, you can find two other example notebooks with model testing for [regression](../regression/regression-TSI.ipynb) and [forecasting](../forecasting/forecasting-TSI.ipynb). Also, these [docs](https://docs.microsoft.com/en-us/azure/machine-learning/concept-automated-ml#training-validation-and-test-data) and [example code snippets](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-configure-cross-validation-data-splits#provide-test-data-preview) can give you more useful information.

This notebook is using remote compute to do all the computation.

If you are using an Azure Machine Learning Compute Instance, you are all set. Otherwise, go through the [configuration](../configuration.ipynb) notebook first if you haven't already to establish your connection to the AzureML Workspace. 

In this notebook you will learn how to:
1. Setup Workspace and Compute Instance.
2. Prepare data and configure AutoML using `AutoMLConfig` to enable test run
3. Retrieve and explore the results.
4. Start remote test runs on demand

## Setup

As part of the setup you have already created an Azure ML `Workspace` object. For Automated ML you will need to create an `Experiment` object, which is a named object in a `Workspace` used to run experiments.

In [ ]:
import logging

from matplotlib import pyplot as plt
import pandas as pd
import os

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.train.automl import AutoMLConfig

This sample notebook may use features that are not available in previous versions of the Azure ML SDK.

In [ ]:
print("This notebook was created using version 1.40.0 of the Azure ML SDK")
print("You are currently using version", azureml.core.VERSION, "of the Azure ML SDK")

In [ ]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = "test-set-support-automl-classification-ccard-remote"

experiment = Experiment(ws, experiment_name)

output = {}
output["Subscription ID"] = ws.subscription_id
output["Workspace"] = ws.name
output["Resource Group"] = ws.resource_group
output["Location"] = ws.location
output["Experiment Name"] = experiment.name
pd.set_option("display.max_colwidth", None)
outputDf = pd.DataFrame(data=output, index=[""])
outputDf.T

A compute target is required to execute the Automated ML run. In this tutorial, you create AmlCompute as your training compute resource.

If the AmlCompute with that name is already in your workspace this code will skip the creation process.
As with other Azure services, there are limits on certain resources (e.g. AmlCompute) associated with the Azure Machine Learning service. Please read [this article](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-manage-quotas) on the default limits and how to request more quota.

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Found existing cluster, use it.")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(
        vm_size="STANDARD_DS12_V2", max_nodes=6
    )
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

## Data

Load the credit card dataset from a csv file containing both training features and labels. The features are inputs to the model, while the training labels represent the expected output of the model. 

In [ ]:
data = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/creditcard.csv"
dataset = Dataset.Tabular.from_delimited_files(data)
label_column_name = "Class"

## Train

Instantiate a AutoMLConfig object. This defines the settings and data used to run the experiment.

|Property|Description|
|-|-|
|**task**|classification or regression|
|**primary_metric**|This is the metric that you want to optimize. Classification supports the following primary metrics: <br><i>accuracy</i><br><i>AUC_weighted</i><br><i>average_precision_score_weighted</i><br><i>norm_macro_recall</i><br><i>precision_score_weighted</i>|
|**enable_early_stopping**|Stop the run if the metric score is not showing improvement.|
|**n_cross_validations**|Number of cross validation splits.|
|**training_data**|Input dataset, containing both features and label column.|
|**label_column_name**|The name of the label column.|

**_You can find more information about primary metrics_** [here](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-configure-auto-train#primary-metric)

### Data Splitting

Next, we'll do train/validation/test split to the data by specifying parameters in `AutoMLConfig`.

You can use `test_size` to determine what percentage of the data to be the test set, which will be used for getting predictions and metrics for the best model after all model training has completed. This parameter must be a floating point value between 0.0 and 1.0. For regression based tasks, random sampling is used. For classification tasks, stratified sampling is used. **Forecasting does not currently support specifying a test dataset using a train/test split**.

**Note that** if `test_size` and `validation_size` are both set, like the example below, the provided dataset will be first splitted into a training set and a test set. Then the training set will be splitted again into a new training set and a validation set. For example, if the original dataset has 1000 samples, `validation_size = 0.2` and `test_size = 0.1`, the final test set will have `1000 * 0.1 = 100` samples, training set will have `1000 * (1 - 0.1) * (1 - 0.2) = 720` samples and validation set will have `1000 * (1 - 0.1) * 0.2 = 180` samples

If you would prefer provide test data directly instead of using a train/test split, use the `test_data` parameter when creating the `AutoMLConfig`.  In the [example notebook for regression](../regression/regression-TSI.ipynb) you can find an example using `test_data`. Test data in the form of a `Dataset` can be passed in to `AutoMLConfig` using the `test_data` parameter. When this parameter is specified, a test run will be started after all training runs have completed and the best model has been identified.

Note, the `test_data` and `test_size` `AutoMLConfig` parameters are mutually exclusive and can not be specified at the same time.

In [ ]:
# Use test_size for train/validation/test split

automl_settings = {
    "primary_metric": "average_precision_score_weighted",
    "enable_early_stopping": True,
    "max_concurrent_iterations": 6,  # This is a limit for testing purpose, please increase it as per cluster size
    "iterations": 10,
    "experiment_timeout_hours": 0.25,  # This is a time limit for testing purposes, remove it for real use cases, this will drastically limit ablity to find the best model possible
    "verbosity": logging.INFO,
}

automl_config = AutoMLConfig(
    task="classification",
    compute_target=compute_target,
    training_data=dataset,
    label_column_name=label_column_name,
    validation_size=0.2,  # Use validation set split. This will be used to select the "winner" model
    test_size=0.1,  # Use test set split. This will be used only for the Winner model Test metrics
    **automl_settings
)

Call the `submit` method on the experiment object and pass the run configuration. Depending on the data and the number of iterations this can run for a while. Validation errors and current status will be shown when setting `show_output=True` and the execution will be synchronous.

In [ ]:
remote_run = experiment.submit(automl_config, show_output=True)

In [ ]:
# If you need to retrieve a run that already started, use the following code
# from azureml.train.automl.run import AutoMLRun
# remote_run = AutoMLRun(experiment = experiment, run_id = '<replace with your run id>')

## Results Analysis and Test

### Widget for Monitoring Runs

The widget will first report a "loading" status while running the first iteration. After completing the first iteration, an auto-updating graph and table will be shown. The widget will refresh once per minute, so you should see the graph update as child runs complete.

**Note:** The widget displays a link at the bottom. Use this link to open a web interface to explore the individual run details

In [ ]:
from azureml.widgets import RunDetails

RunDetails(remote_run).show()

In [ ]:
remote_run.wait_for_completion(show_output=False)

### Explain model

Automated ML models can be explained and visualized using the SDK Explainability library. 

### Retrieve the Best Model/Run

Below we select the best pipeline from our iterations. The `get_output` method returns the best run and the fitted model.  Overloads on `get_output` allow you to retrieve the best run and fitted model for *any* logged metric or for a particular *iteration*.

In [ ]:
best_run, fitted_model = remote_run.get_output()
fitted_model

#### Get the Predictions and Metrics Generated by the Test Run

Get the test run associated with the best run. When a remote test run is requested by passing in a value for `test_data` or `test_size` to `AutoMLConfig`, only the best training run will have a test run associated with it. To start a test run for models which are not associated with the best run or to start another best run test run, use `ModelProxy`. See the example later in the notebook in the __Test the fitted model remotely__ section for more details.

To see more details about the test run in Azure Machine Learning Studio (view its metrics, get a preview of the predictions, etc...) follow the link to the __Details Page__ listed in the next cells output.

In [ ]:
test_run = next(best_run.get_children(type="automl.model_test"))
test_run.wait_for_completion(show_output=False, wait_post_processing=True)
test_run

Get the __metrics__ from the test run.

In [ ]:
test_run_metrics = test_run.get_metrics()
for name, value in test_run_metrics.items():
    print(f"{name}: {value}")

Get the __predictions__ from the test run.

In [ ]:
test_run_details = test_run.get_details()
test_run_predictions = Dataset.get_by_id(
    ws, test_run_details["outputDatasets"][0]["identifier"]["savedId"]
)
test_run_predictions.to_pandas_dataframe().head()

### Test the fitted model remotely (On-demand)
This is testing a model the same way it was tested at the end of the training. However, in this case it's performed on-demand, meaning that you can test the model at any time, not just right after training.

In [ ]:
from azureml.train.automl.model_proxy import ModelProxy

# get test data from the full dataset
training_data, test_data = dataset.random_split(percentage=0.8, seed=223)

model_proxy = ModelProxy(best_run)
predictions, test_run_metrics = model_proxy.test(test_data)

predictions.to_pandas_dataframe().head()

In [ ]:
pd.DataFrame.from_dict(test_run_metrics, orient="index", columns=["Value"])

## Acknowledgements

This Credit Card fraud Detection dataset is made available under the Open Database License: http://opendatacommons.org/licenses/odbl/1.0/. Any rights in individual contents of the database are licensed under the Database Contents License: http://opendatacommons.org/licenses/dbcl/1.0/ and is available at: https://www.kaggle.com/mlg-ulb/creditcardfraud

The dataset has been collected and analysed during a research collaboration of Worldline and the Machine Learning Group (http://mlg.ulb.ac.be) of ULB (Université Libre de Bruxelles) on big data mining and fraud detection.
More details on current and past projects on related topics are available on https://www.researchgate.net/project/Fraud-detection-5 and the page of the DefeatFraud project

Please cite the following works:

Andrea Dal Pozzolo, Olivier Caelen, Reid A. Johnson and Gianluca Bontempi. Calibrating Probability with Undersampling for Unbalanced Classification. In Symposium on Computational Intelligence and Data Mining (CIDM), IEEE, 2015

Dal Pozzolo, Andrea; Caelen, Olivier; Le Borgne, Yann-Ael; Waterschoot, Serge; Bontempi, Gianluca. Learned lessons in credit card fraud detection from a practitioner perspective, Expert systems with applications,41,10,4915-4928,2014, Pergamon

Dal Pozzolo, Andrea; Boracchi, Giacomo; Caelen, Olivier; Alippi, Cesare; Bontempi, Gianluca. Credit card fraud detection: a realistic modeling and a novel learning strategy, IEEE transactions on neural networks and learning systems,29,8,3784-3797,2018,IEEE

Dal Pozzolo, Andrea Adaptive Machine learning for credit card fraud detection ULB MLG PhD thesis (supervised by G. Bontempi)

Carcillo, Fabrizio; Dal Pozzolo, Andrea; Le Borgne, Yann-Aël; Caelen, Olivier; Mazzer, Yannis; Bontempi, Gianluca. Scarff: a scalable framework for streaming credit card fraud detection with Spark, Information fusion,41, 182-194,2018,Elsevier

Carcillo, Fabrizio; Le Borgne, Yann-Aël; Caelen, Olivier; Bontempi, Gianluca. Streaming active learning strategies for real-life credit card fraud detection: assessment and visualization, International Journal of Data Science and Analytics, 5,4,285-300,2018,Springer International Publishing

Bertrand Lebichot, Yann-Aël Le Borgne, Liyun He, Frederic Oblé, Gianluca Bontempi Deep-Learning Domain Adaptation Techniques for Credit Cards Fraud Detection, INNSBDDL 2019: Recent Advances in Big Data and Deep Learning, pp 78-88, 2019

Fabrizio Carcillo, Yann-Aël Le Borgne, Olivier Caelen, Frederic Oblé, Gianluca Bontempi Combining Unsupervised and Supervised Learning in Credit Card Fraud Detection Information Sciences, 2019